# Encryption and Cryptography

Python doesnt have native encryption tools, instead it provides hashing. Hashing + 3rd party Encryption and Cryptography pakcages will be covered in this chapter.

## Hashing

One of the most common uses of hasdhing is to store a hash of a password as opposed to the password itself. The hash must be legit or it can be decrypted. It is also used for sending a hash of a file alongside a file so the reciever can compare the two and confirm it has not been modified by an outside source.

In [1]:
# md5 hash

import hashlib
md5 = hashlib.md5()

# Unicode must be byte encoded before hashing
hash_str = b'I am going to hash this string!'
md5.update(hash_str)

print(md5.digest())

b'l\x1e\xfb\x00U\x14\xcdT\xf7\x1c\xf3l\xcd&o\x10'


In [2]:
# printing a hex digest instead
print(md5.hexdigest())

6c1efb005514cd54f71cf36ccd266f10


In [3]:
# Creating a shai512 hash
# Less lines of code, shorter hash = less secure

import hashlib
sha = hashlib.sha1(b'Hello World!').hexdigest()

print(sha)

2ef7bde608ce5404e97d5f042f95f89f1c232871


## Key Derivation

In [4]:
# using pbkdf2_hmac for encryption it uses a psuedo-ransdom function making it5 harder to decode
import hashlib
import binascii


dk = hashlib.pbkdf2_hmac(hash_name='sha256',
        password=b'rando_password', 
        salt=b'bad_salt', 
        iterations=1000000)

print (binascii.hexlify(dk))

b'0ea6b4654028f2b73cca77497c8d9dc4f7c712069d2098fb84b727c6f4d05e79'


## PyCrypto

PyCrypto is the most well known 3rd party package for ecryption in python.

In [5]:
# Installing pycryptodome, a drop-in replacement for pycrypto

!pip install pycryptodome

In [6]:
## encrypting a string

from Crypto.Cipher import DES
key = b'abcdefgh'
def pad(text):
        while len(text) % 8 != 0:
            text += b' '
        return text
des = DES.new(key, DES.MODE_ECB)
text = b'Python rocks!'
padded_text = pad(text)

encrypted_text = des.encrypt(padded_text)
print (encrypted_text)

b'>\xfc\x1f\x16x\x87\xb2\x93\x0e\xfcH\x02\xd59VQ'


In [7]:
# decrypting our encrpyted string
print(des.decrypt(encrypted_text))

b'Python rocks!   '


In [8]:
# Encrypting data with RSA
# Can use a public/private RSA key or create your own, here we weill create one

from Crypto.PublicKey import RSA
code = 'nooneknows'
key = RSA.generate(2048)
encrypted_key = key.exportKey(passphrase=code, pkcs=8, 
        protection="scryptAndAES128-CBC")
with open('14_encrpytion_and_crpytography_demos/my_private_rsa_key.bin', 'wb') as f:
        f.write(encrypted_key)
with open('14_encrpytion_and_crpytography_demos/my_rsa_public.pem', 'wb') as f:
        f.write(key.publickey().exportKey())

In [9]:
# Encrpyting a File!

from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES, PKCS1_OAEP

with open('14_encrpytion_and_crpytography_demos/encrypted_data.bin', 'wb') as out_file:
    recipient_key = RSA.import_key(
        open('14_encrpytion_and_crpytography_demos/my_rsa_public.pem').read())
    session_key = get_random_bytes(16)

    cipher_rsa = PKCS1_OAEP.new(recipient_key)
    out_file.write(cipher_rsa.encrypt(session_key))

    cipher_aes = AES.new(session_key, AES.MODE_EAX)
    data = b'blah blah blah Python blah blah'
    ciphertext, tag = cipher_aes.encrypt_and_digest(data)

    out_file.write(cipher_aes.nonce)
    out_file.write(tag)
    out_file.write(ciphertext)

In [10]:
# Decrypting our file data

from Crypto.PublicKey import RSA
from Crypto.Cipher import AES, PKCS1_OAEP

code = 'nooneknows'

with open('14_encrpytion_and_crpytography_demos/encrypted_data.bin', 'rb') as fobj:
    private_key = RSA.import_key(
        open('14_encrpytion_and_crpytography_demos/my_private_rsa_key.bin').read(),
        passphrase=code)

    enc_session_key, nonce, tag, ciphertext = [ fobj.read(x) 
                                                for x in (private_key.size_in_bytes(), 
                                                16, 16, -1) ]

    cipher_rsa = PKCS1_OAEP.new(private_key)
    session_key = cipher_rsa.decrypt(enc_session_key)

    cipher_aes = AES.new(session_key, AES.MODE_EAX, nonce)
    data = cipher_aes.decrypt_and_verify(ciphertext, tag)

print(data)

b'blah blah blah Python blah blah'


## The cryptography package

The cryptography package aims to be “cryptography for humans” much like the requests library is “HTTP for Humans”.

In [11]:
# Installing library
!pip install cryptography

In [15]:
# Using the Fernet module to create a message that cannot be read or manipulated without the key we define

from cryptography.fernet import Fernet
cipher_key = Fernet.generate_key()
print(cipher_key)

cipher = Fernet(cipher_key)
text = b'My super secret message'
encrypted_text = cipher.encrypt(text)
print(encrypted_text)

decrypted_text = cipher.decrypt(encrypted_text)
print(decrypted_text)

b'vEndXwJuXISCAJdSWYMCtpTFaENKVYr-IxWNBgOOfCc='
b'gAAAAABhmprT2ElcMvMZ8iVaSZZx2TX-LRYiMAqVvnQ2ay9cvC7vK_Sho7IOKvWTAWpik8Z07BcyN2BT4gYm-MnBxKzxdqpyLO-8Ru8IJo2TTMnk5GAdF8Y='
b'My super secret message'
